# 02 Construction des features

Ce notebook a pour objectif de transformer les données prétraitées en features exploitables par les modèles de recommandation. Nous construisons :

- **Matrice d'interaction** user–item sparse pondérée et normalisée.
- **Cartographies** des identifiants utilisateurs et vidéos.
- **Features additionnels** : nombre total d'interactions par utilisateur, moyenne quotidienne de vues par vidéo.

Les fichiers générés seront sauvegardés dans le dossier `features/` pour être réutilisés dans Model_Development

## 1. Imports

In [8]:
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import normalize
import joblib
import numpy as np
import os

## 2. Fonction `build_interaction_matrix`

Cette fonction construit la matrice user–item pondérée selon :

1. **Filtrage** : on conserve uniquement les utilisateurs et vidéos ayant au moins `min_interactions`.
2. **Mapping** : on crée deux dictionnaires `user_map` et `video_map` pour indexer les IDs.
3. **Pondération** :
   - `play_duration_s` (durée de vue en secondes)
   - **Source weight** (`alpha`) pour ajuster l’importance globale des interactions.
   - **Décroissance temporelle** : poids exponentiel selon l’âge de l’interaction (`decay`).
4. **Construction** : on assemble un `csr_matrix` et on normalise chaque ligne (L1) pour obtenir des profils comparables.

**Arguments** :
- `train_df` : DataFrame d’interactions contenant `user_id`, `video_id`, `play_duration`, `timestamp`.
- `min_interactions` : seuil minimal pour filtrer.
- `alpha` : coefficient de pondération source.
- `decay` : taux de décroissance exponentielle temporelle.

**Retour** :
- `mat` : matrice user–item normalisée (csr_matrix)
- `user_map`, `video_map` : dicts pour passer des IDs crues aux indices de la matrice

In [9]:
def build_interaction_matrix(train_df, min_interactions=5, alpha=1.0, decay=1e-7):
    # Copie pour ne pas modifier l'original
    df = train_df.copy()
    # Poids constant pour chaque interaction (peut intégrer d'autres sources)
    df["source_weight"] = alpha

    # Filtrage des utilisateurs et vidéos trop rares
    active_users  = df["user_id"].value_counts()[lambda x: x >= min_interactions].index
    active_videos = df["video_id"].value_counts()[lambda x: x >= min_interactions].index
    df = df[df["user_id"].isin(active_users) & df["video_id"].isin(active_videos)]

    # Création des mappings id -> index
    user_map  = {u: i for i, u in enumerate(df["user_id"].unique())}
    video_map = {v: k for k, v in enumerate(df["video_id"].unique())}
    df["uidx"] = df["user_id"].map(user_map)
    df["vidx"] = df["video_id"].map(video_map)

    # Conversion de la durée de lecture en secondes
    df["play_duration_s"] = df["play_duration"] / 1000.0

    # Calcul de l'âge de l'interaction pour la décroissance temporelle
    max_ts = df["timestamp"].astype(int).max() / 1e9
    df["age"] = (max_ts - df["timestamp"].astype(int) / 1e9)
    df["time_weight"] = np.exp(-decay * df["age"])

    # Force des interactions : durée * source_weight * time_weight
    df["strength"] = df["play_duration_s"] * df["source_weight"] * df["time_weight"]

    # Assemblage de la matrice creuse
    rows = df["uidx"].to_numpy()
    cols = df["vidx"].to_numpy()
    data = df["strength"].to_numpy()
    mat = sp.csr_matrix((data, (rows, cols)), shape=(len(user_map), len(video_map)))

    # Normalisation L1 par ligne
    mat = normalize(mat, norm="l1", axis=1)
    return mat, user_map, video_map


## 3. Chargement des données et construction de la matrice
On charge la matrice complète prétraitée (`big_matrix`) et décide de construire la matrice d'interaction.

In [10]:
big = pd.read_parquet("preprocessed/big_matrix.parquet")
daily = pd.read_parquet("preprocessed/item_daily_features.parquet")

# Construction de la matrice sparse et des mappings
mat, user_map, video_map = build_interaction_matrix(big, alpha=1.0)

## 4. Sauvegarde des features

Pour réutiliser rapidement ces objets, on les enregistre dans `features/` :
- Matrice d'interaction au format `.npz` (scipy)
- Mappings utilisateur/vidéo au format `joblib`
- Features additionnels calculés par agrégation

In [11]:
os.makedirs("features", exist_ok=True)

# Sauvegarde de la matrice d'interaction et des mappings
sp.save_npz("features/interaction_matrix.npz", mat)
joblib.dump(user_map, "features/user_map.pkl")
joblib.dump(video_map, "features/video_map.pkl")

# Feature : nombre total d'interactions par utilisateur
df_users = big.groupby("user_id").size().rename("total_interactions").reset_index()
df_users.to_parquet("features/user_features.parquet", index=False)

# Feature : moyenne quotidienne de vues par vidéo
# Agrégation de 'play_cnt' dans daily
df_videos = daily.groupby("video_id")["play_cnt"].mean().rename("avg_daily_plays").reset_index()
df_videos.to_parquet("features/video_features.parquet", index=False)

print("Features sauvegardés dans features/")

Features sauvegardés dans features/
